In [1]:
pip install requests tqdm

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from pathlib import Path
from tqdm import tqdm
import zipfile
import os
import json

# Where to save data
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

# Load your Kaggle credentials
#with open(Path.home() / ".kaggle/kaggle.json") as f:
    #creds = json.load(f)

KAGGLE_USERNAME = "vaishnavibodla"
KAGGLE_KEY = "6d1f311e4fe7b274e982ff995024378d"

def download_file(file_name):
    url = f"https://www.kaggle.com/api/v1/competitions/data/download/amex-default-prediction/{file_name}"
    auth = (KAGGLE_USERNAME, KAGGLE_KEY)
    headers = {"User-Agent": "Mozilla/5.0"}
    zip_path = DATA_DIR / f"{file_name}.zip"

    print(f"⬇️  Downloading {file_name}...")

    with requests.get(url, stream=True, auth=auth, headers=headers) as r:
        r.raise_for_status()
        total_size = int(r.headers.get("content-length", 0))
        with open(zip_path, "wb") as f:
            for chunk in tqdm(r.iter_content(chunk_size=1024 * 1024),  # 1MB chunks
                              total=total_size // (1024 * 1024),
                              unit="MB"):
                f.write(chunk)

    print(f"✅ Downloaded {file_name}. Extracting...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)

    os.remove(zip_path)
    print(f"🧹 Cleaned up {file_name}.zip")

# Files to grab
files = ["train_data.csv", "train_labels.csv", "sample_submission.csv"]
for f in files:
    download_file(f)


⬇️  Downloading train_data.csv...


6827MB [01:45, 64.91MB/s]                          


✅ Downloaded train_data.csv. Extracting...
🧹 Cleaned up train_data.csv.zip
⬇️  Downloading train_labels.csv...


17MB [00:00, 41.92MB/s]                        


✅ Downloaded train_labels.csv. Extracting...
🧹 Cleaned up train_labels.csv.zip
⬇️  Downloading sample_submission.csv...


33MB [00:00, 60.73MB/s]                        


✅ Downloaded sample_submission.csv. Extracting...
🧹 Cleaned up sample_submission.csv.zip


In [3]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

input_file = "data/train_data.csv"
chunksize = 200_000  # adjust based on RAM

writer = None

for chunk in pd.read_csv(input_file, chunksize=chunksize):
    # Reduce precision
    for col in chunk.select_dtypes(include=['float64']).columns:
        chunk[col] = chunk[col].astype('float32')
    for col in chunk.select_dtypes(include=['int64']).columns:
        chunk[col] = chunk[col].astype('int32')

    # Convert to Arrow table
    table = pa.Table.from_pandas(chunk)

    # Write in append mode
    if writer is None:
        writer = pq.ParquetWriter("final_dataset.parquet", table.schema, compression="zstd")
    writer.write_table(table)

if writer:
    writer.close()

print("✅ Done! final_dataset.parquet saved")

✅ Done! final_dataset.parquet saved
